In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [3]:


def evaluate_label_changes_vectorized(start_epoch, end_epoch, ratio=True, csv_dir="."):
    """
    指定したエポック範囲内のCSVファイルから、各サンプルの予測ラベルの変化数をベクトル演算で高速に計算する関数。
    
    各CSVファイルは "test_inferenc_epoch_{epoch}.csv" という名前で存在するものとする。
    読み込む際は、"TestData_Index" と "Predicted_Label" の2列のみを対象とする。

    Parameters
    ----------
    start_epoch : int
        評価開始エポック
    end_epoch : int
        評価終了エポック
    ratio : bool, default True
        Trueの場合、変化数を (エポック数 - 1) で割った比率として返す。
        Falseの場合は変化数そのままを返す。
    csv_dir : str, default "."
        CSVファイルが存在するディレクトリのパス

    Returns
    -------
    pd.DataFrame or None
        DataFrameは "TestData_Index" と "Label_Change" の列を持つ。
        十分なデータが取得できなかった場合は None を返す。
    """
    predictions_list = []
    test_data_index = None
    epochs_found = []

    for epoch in range(start_epoch, end_epoch + 1):
        csv_filename = os.path.join(csv_dir, f"test_inferenc_epoch_{epoch}.csv")
        if not os.path.exists(csv_filename):
            print(f"Warning: {csv_filename} が存在しません。epoch {epoch} はスキップします。")
            continue

        try:
            # 必要な2列のみ読み込む
            df = pd.read_csv(csv_filename, usecols=["TestData_Index", "Predicted_Label"])
        except Exception as e:
            print(f"Error: {csv_filename} の読み込みに失敗しました。{e}")
            continue

        if test_data_index is None:
            # 最初に読み込んだ TestData_Index を基準として保持
            test_data_index = df["TestData_Index"].sort_values().reset_index(drop=True)
        else:
            # すでに読み込んだ test_data_index と一致しているかチェック（ソート済みである前提）
            current_index = df["TestData_Index"].sort_values().reset_index(drop=True)
            if not test_data_index.equals(current_index):
                print(f"Warning: {csv_filename} の TestData_Index が基準と一致しません。")
                continue

        # TestData_Index でソートして Predicted_Label のみ取得
        predictions_list.append(df.sort_values("TestData_Index").reset_index(drop=True)["Predicted_Label"])
        epochs_found.append(epoch)

    if len(predictions_list) < 2:
        print("評価するための十分なエポックのCSVファイルが読み込めませんでした。")
        return None

    # 各エポックの予測ラベルを1つの DataFrame にまとめる（各列が1エポック分）
    pred_df = pd.concat(predictions_list, axis=1)
    pred_df.columns = [f"epoch_{e}" for e in epochs_found]

    # 隣接するエポック間でラベルが変わっているかを一括で判定
    # diff() は最初の列でNaNになるので、その後の列について比較する
    changes = (pred_df.diff(axis=1).iloc[:, 1:] != 0).astype(int)
    change_counts = changes.sum(axis=1)
    num_transitions = changes.shape[1]

    if ratio and num_transitions > 0:
        label_change = change_counts / num_transitions
    else:
        label_change = change_counts

    result_df = pd.DataFrame({
        "TestData_Index": test_data_index,
        "Label_Change": label_change
    })

    return result_df

def evaluate_label_changes_for_ranges_vectorized(epoch_ranges, ratio=True, csv_dir="."):
    """
    複数のエポック範囲に対して、ベクトル演算を利用した評価を行い、その結果を
    処理対象のCSVファイルが存在するフォルダの親ディレクトリ直下に作成した temporal_stability フォルダ内に、
    各エポック範囲ごとにファイル名に範囲情報を含めたCSVとして保存する関数。

    Parameters
    ----------
    epoch_ranges : list of tuple
        各タプル (start_epoch, end_epoch) の形式でエポック範囲を指定。例: [(1, 1000), (1001, 2000)]
    ratio : bool, default True
        True の場合、変化数を (エポック数 - 1) で割った比率として計算する。
        False の場合は変化数そのままを使用する。
    csv_dir : str, default "."
        各エポックのCSVファイルが存在するディレクトリのパス

    Returns
    -------
    pd.DataFrame or None
        複数範囲の結果をまとめた DataFrame を返す。十分なデータがなければ None を返す。
    """
    # csv_dir の親ディレクトリ直下に temporal_stability フォルダを作成
    parent_dir = os.path.dirname(os.path.abspath(csv_dir))
    output_dir = os.path.join(parent_dir, "temporal_stability")
    os.makedirs(output_dir, exist_ok=True)

    combined_results = []
    for start_epoch, end_epoch in epoch_ranges:
        print(f"Evaluating epoch range: {start_epoch} - {end_epoch}")
        df_result = evaluate_label_changes_vectorized(start_epoch, end_epoch, ratio=ratio, csv_dir=csv_dir)
        if df_result is None:
            print(f"Epoch range {start_epoch}-{end_epoch} の評価結果が得られませんでした。")
            continue

        # 結果に範囲情報を追加（任意）
        df_result["Epoch_Range"] = f"{start_epoch}-{end_epoch}"
        # ファイル名にエポック範囲を含めたパス
        output_file = os.path.join(output_dir, f"label_change_{start_epoch}-{end_epoch}.csv")
        try:
            df_result.to_csv(output_file, index=False)
            print(f"Epoch range {start_epoch}-{end_epoch} の結果を保存しました: {output_file}")
        except Exception as e:
            print(f"CSV の出力に失敗しました: {e}")
        combined_results.append(df_result)

    if combined_results:
        final_df = pd.concat(combined_results, ignore_index=True)
        combined_output_file = os.path.join(output_dir, "combined_label_changes_raw.csv")
        try:
            final_df.to_csv(combined_output_file, index=False)
            print(f"全範囲の結果をまとめたCSVを保存しました: {combined_output_file}")
        except Exception as e:
            print(f"Combined CSV の出力に失敗しました: {e}")
        return final_df
    else:
        print("有効な評価結果が得られませんでした。")
        return None

In [4]:
# 利用例:
if __name__ == "__main__":
    # 複数のエポック範囲で評価を実施（例：1～1000 と 1001～2000）
    epoch_ranges = [(1,30),(30,60),(60,140),(140,1000)]
    # csv_dir には処理対象のCSVファイルが格納されているディレクトリのパスを指定
    result_df = evaluate_label_changes_for_ranges_vectorized(epoch_ranges, ratio=False, csv_dir="/workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/csv")


Evaluating epoch range: 1 - 30
Epoch range 1-30 の結果を保存しました: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/label_change_1-30.csv
Evaluating epoch range: 30 - 60
Epoch range 30-60 の結果を保存しました: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/label_change_30-60.csv
Evaluating epoch range: 60 - 140
Epoch range 60-140 の結果を保存しました: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/label_change_60-140.csv
Evaluating epoch range: 140 - 1000
Epoch range 140-1000 の結果を保存しました: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_Lab

In [20]:
def visualize_label_change_distribution(folder_path, epoch_ranges=None):
    """
    指定フォルダ内の 'label_change_[学習区間].csv' 形式のファイルを対象に、
    各CSVごとに "Label_Change" の分布をヒストグラムで可視化し、学習区間をファイル名に含めて保存。

    改良点:
    - X軸とY軸のメモリを適切に設定（自動調整）
    - 背景グリッドを追加
    - 視認性の高いデザインを適用

    Parameters
    ----------
    folder_path : str
        CSVファイルが格納されているフォルダのパス
    epoch_ranges : list of str, optional
        処理する特定の学習区間のリスト（例: ["0-100", "101-200"]）。指定しない場合はすべてのファイルを処理。
    """
    
    # 'label_change_[学習区間].csv' 形式のファイルをリストアップ
    csv_files = [f for f in os.listdir(folder_path) if re.match(r"label_change_\d+-\d+\.csv", f)]
    if not csv_files:
        print(f"対象のCSVファイルが見つかりません: {folder_path}")
        return

    for csv_file in csv_files:
        # 学習区間をファイル名から抽出
        match = re.search(r"label_change_(\d+-\d+)", csv_file)
        file_epoch_range = match.group(1) if match else "unknown"

        # 特定の学習区間が指定されている場合、その区間のみ処理
        if epoch_ranges and file_epoch_range not in epoch_ranges:
            continue

        file_path = os.path.join(folder_path, csv_file)
        print(f"処理中: {file_path}")

        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"CSV 読み込み失敗: {file_path}, エラー: {e}")
            continue

        # ======== 1) Label_Change 列のヒストグラム ========
        if "Label_Change" not in df.columns:
            print(f"{csv_file} に 'Label_Change' 列がありません。スキップします。")
            continue

        # ヒストグラムを作成
        plt.figure(figsize=(8, 6))
        plt.hist(df["Label_Change"], bins = np.arange(0, 31, 1), color="blue", alpha=0.7, edgecolor="black")

        # 軸ラベルの設定
        plt.xlabel("Temporal instability", fontsize=14, fontweight="bold")
        plt.ylabel("Frequency", fontsize=14, fontweight="bold")

        # タイトルの設定
        plt.title(f"Distribution of temporal instability\nEpoch Range: {file_epoch_range}", fontsize=16, fontweight="bold")
         # 0から1まで0.2ずつ目盛りを設定
        plt.yticks(fontsize=12)
        plt.ylim(0, 40000)
        plt.xlim(0, 30)

        # 図をファイルに保存（"hist_label_change_[学習区間].png" として出力）
        output_hist_path = os.path.join(folder_path, f"hist_label_change_{file_epoch_range}_raw.png")
        plt.savefig(output_hist_path, dpi=300, bbox_inches="tight", facecolor='white')
        plt.close()
        print(f"  => Label Change ヒストグラム保存: {output_hist_path}")



In [21]:
# -----------------------------
# メイン実行部
# -----------------------------
if __name__ == "__main__":
    # temporal_stability フォルダのパスを指定
    print("start")
    folder_path = "/workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability"
    epoch_range = ["1-30","30-60","60-140","140-1000"]  # 処理したい特定の学習区間を指定
    visualize_label_change_distribution(folder_path, epoch_range)

start
処理中: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/label_change_1-30.csv


  => Label Change ヒストグラム保存: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/hist_label_change_1-30_raw.png
処理中: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/label_change_30-60.csv
  => Label Change ヒストグラム保存: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/hist_label_change_30-60_raw.png
処理中: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/label_change_140-1000.csv
  => Label Change ヒストグラム保存: /workspace/test_inference/seed_42width4_cnn_5layers_distributi

In [21]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

def plot_individual_and_combined_temporal_instability(folder_path):
    """
    指定フォルダ内の 'label_change_[学習区間].csv' 形式のファイルについて、
    1. 各CSVごとに、横軸を TestData_Index、縦軸を Label_Change とした個別の散布図を作成・保存。
    2. すべての学習区間の散布図を一つの図にまとめる。  
       ※色は、学習区間の開始値が小さいものが青、開始値が大きくなるにつれて赤のグラデーションになるように設定。
    ※ 全グラフの y 軸は 0～1 に固定しています。
    
    Parameters
    ----------
    folder_path : str
        CSVファイルが格納されているフォルダのパス（例: "/mnt/data/temporal_stability"）
    """
    # 対象のCSVファイルのみをリストアップ
    pattern = r"label_change_(\d+)-(\d+)\.csv"
    csv_files = [f for f in os.listdir(folder_path) if re.match(pattern, f)]
    if not csv_files:
        print(f"指定フォルダ内に対象のCSVファイルが見つかりません: {folder_path}")
        return

    # 各CSVファイルから学習区間の開始値を取得し、辞書にまとめる
    interval_data = {}  # key: csv filename, value: (start, end, dataframe)
    start_values = []
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        m = re.match(pattern, csv_file)
        if m:
            start, end = int(m.group(1)), int(m.group(2))
            try:
                df = pd.read_csv(file_path)
            except Exception as e:
                print(f"CSV読み込み失敗: {file_path} エラー: {e}")
                continue

            if "TestData_Index" not in df.columns or "Label_Change" not in df.columns:
                print(f"{csv_file} に必要なカラムがありません。スキップします。")
                continue

            interval_data[csv_file] = (start, end, df)
            start_values.append(start)

    if not interval_data:
        print("有効なデータがありませんでした。")
        return

    # 出力先フォルダ "fig" を作成（存在しない場合）
    output_dir = os.path.join(folder_path, "fig")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 個別プロットの作成
    for csv_file, (start, end, df) in interval_data.items():
        plt.figure(figsize=(10, 6))
        plt.scatter(df["TestData_Index"], df["Label_Change"], color="blue", alpha=0.7, s=10)
        plt.xlabel("TestData_Index", fontsize=14, fontweight="bold")
        plt.ylabel("Label Change (Temporal Instability)", fontsize=14, fontweight="bold")
        plt.title(f"Temporal Instability\nEpoch Range: {start}-{end}", fontsize=16, fontweight="bold")
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.ylim(0, 1)
        output_path = os.path.join(output_dir, f"scatter_temporal_instability_{start}-{end}.png")
        plt.savefig(output_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"個別図保存: {output_path}")

    # --- すべての学習区間をまとめた図の作成 ---
    # カラーマップの設定（開始値が小さい→青、大きい→赤のグラデーション）
    norm = Normalize(vmin=min(start_values), vmax=max(start_values))
    colormap = cm.get_cmap("coolwarm")  # coolwarm: 青～赤のグラデーション

    plt.figure(figsize=(12, 8))
    for csv_file, (start, end, df) in sorted(interval_data.items(), key=lambda x: x[1][0]):
        color = colormap(norm(start))
        label = f"{start}-{end}"
        plt.scatter(df["TestData_Index"], df["Label_Change"], color=color, label=label, alpha=0.8, s=10)
    plt.xlabel("TestData_Index", fontsize=14, fontweight="bold")
    plt.ylabel("Label Change (Temporal Instability)", fontsize=14, fontweight="bold")
    plt.title("Temporal Instability Comparison Across Epoch Ranges", fontsize=16, fontweight="bold")
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylim(0, 1)

    plt.legend(title="Epoch Range", fontsize=10, title_fontsize=12)
    combined_output_path = os.path.join(output_dir, "combined_temporal_instability.png")
    plt.savefig(combined_output_path, dpi=300, bbox_inches="tight", facecolor='white')
    plt.close()
    print(f"すべての学習区間をまとめた図保存: {combined_output_path}")

# -----------------------------
# メイン実行部
# -----------------------------
if __name__ == "__main__":
    folder_path = "/workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability"
    plot_individual_and_combined_temporal_instability(folder_path=folder_path)


個別図保存: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/fig/scatter_temporal_instability_1-30.png
個別図保存: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/fig/scatter_temporal_instability_30-60.png
個別図保存: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/fig/scatter_temporal_instability_140-1000.png
個別図保存: /workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability/fig/scatter_temporal_instability_60-140.png
すべての学習区間をまとめた図保存: /workspace/test_inference/seed_42width4_cnn_5la

In [ ]:
# -----------------------------
# メイン実行部（例として）
# -----------------------------
if __name__ == "__main__":
    # temporal_stabilityフォルダのパスを指定
    folder_path = "/workspace/test_inference/seed_42width4_cnn_5layers_distribution_colored_emnist_variance0_combined_lr0.01_batch256_epoch1000_LabelNoiseRate0.2_Optimsgd_Momentum0.0/temporal_stability"
    analyze_label_changes_for_all_csv(folder_path)